<a href="https://colab.research.google.com/github/xprilion/gemini-function-calling-workshop/blob/main/GeminiFunctionCalling_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Gemini Function Calling

This notebook follows along the required code to complete the lab on [How to Interact with APIs Using Function Calling in Gemini
](https://codelabs.developers.google.com/codelabs/gemini-function-calling) by [@koverholt](https://github.com/koverholt).

## Step 1. Overview
This section talks about the importance of Function Calling and how it enables Gemini to access data that may be real-time, protected or otherwise unavailable in the Gemini training datasets.

## Step 2. Setup and requirements

In [2]:
%%capture
!pip install --upgrade google-cloud-aiplatform

## Step 3. Understand the problem

In [25]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel

In [26]:
from google.oauth2 import service_account

In [27]:
credentials = service_account.Credentials.from_service_account_file('gcp-adventure-x-a3fb7a36e1e6.json')

In [28]:
vertexai.init(project="gcp-adventure-x", location="us-central1", credentials=credentials)

In [29]:
model = GenerativeModel("gemini-1.0-pro-001")

In [30]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

I do not have access to real-time information and cannot provide the current exchange rate. Please check a currency converter or financial news source for the most up-to-date information.


## Step 4: Try common workarounds
Skipping this section in interest of getting ideas from the workshop participants.

## Step 5. How function calling works
This section describes the execution flow of Function Calling.

## Step 6. Choose your API

In [32]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
data = response.json()
data

{'amount': 1.0,
 'base': 'EUR',
 'date': '2024-03-15',
 'rates': {'AUD': 1.6579,
  'BGN': 1.9558,
  'BRL': 5.4461,
  'CAD': 1.4731,
  'CHF': 0.9613,
  'CNY': 7.838,
  'CZK': 25.166,
  'DKK': 7.4571,
  'GBP': 0.8541,
  'HKD': 8.5199,
  'HUF': 393.2,
  'IDR': 17011,
  'ILS': 3.9811,
  'INR': 90.26,
  'ISK': 148.9,
  'JPY': 162.03,
  'KRW': 1448.71,
  'MXN': 18.1915,
  'MYR': 5.1241,
  'NOK': 11.5205,
  'NZD': 1.786,
  'PHP': 60.494,
  'PLN': 4.2953,
  'RON': 4.9711,
  'SEK': 11.2674,
  'SGD': 1.4562,
  'THB': 39.053,
  'TRY': 35.092,
  'USD': 1.0892,
  'ZAR': 20.352}}

## Step 7. Define a function and tool

In [33]:
from vertexai.preview.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.0-pro-001")

In [34]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

In [35]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

## Step 8. Generate a function call

In [57]:
prompt = """What is the latest exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [59]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
    }
  }
}



## Step 9. Make an API request

In [60]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'from': 'AUD', 'date': 'latest', 'to': 'SEK'}

In [62]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.json()

{'amount': 1.0, 'base': 'AUD', 'date': '2024-03-15', 'rates': {'SEK': 6.7962}}

## Step 10. Generate a response

In [63]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

'500 Australian dollars is worth 3398.1 Swedish krona as of 2024-03-15. The exchange rate is 1 AUD = 6.7962 SEK.'

## Conclusion

Explore further:


1.   [Gemini Pro API Reference](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini#gemini-pro)
2.   [Sample Function Calling Notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/function-calling/intro_function_calling.ipynb)

